# Imports

In [2]:
import librosa

import numpy as np
import pandas as pd
import random

import torch
import torchmetrics
import os
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from joblib import Parallel, delayed

import tensorflow as tf

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

### Config

In [32]:
class Config:
    SR = 32000
    N_MFCC = 40
    
    # Dataset
    ROOT_DIR = 'C:/HongBeomsun/Dataset_SSD/FakeVoice'
    
    # Training
    N_CLASSES = 2
    BATCH_SIZE = 96
    N_EPOCHS = 50
    LEARNING_RATE = 1e-3
    
    # Others
    SEED = 42
    
CONFIG = Config()

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [6]:
seed_everything(CONFIG.SEED)

### Data

In [7]:
df = pd.read_csv(os.path.join(CONFIG.ROOT_DIR,'train.csv'))

In [8]:
print(len(df))
df.head()

55438


,id,path,label
0,RUNQPNJF,./train/RUNQPNJF.ogg,real
1,JFAWUOGJ,./train/JFAWUOGJ.ogg,fake
2,RDKEKEVX,./train/RDKEKEVX.ogg,real
3,QYHJDOFK,./train/QYHJDOFK.ogg,real
4,RSPQNHAO,./train/RSPQNHAO.ogg,real


In [9]:
df['label'].value_counts()

label
fake    27818
real    27620
Name: count, dtype: int64

In [10]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CONFIG.SEED, stratify=df['label'])

In [11]:
train['label'].value_counts()
val['label'].value_counts()

label
fake    5564
real    5524
Name: count, dtype: int64

In [1]:
def normalize_volume(y, target_dB=-20):
    rms = np.sqrt(np.mean(y**2))
    loudness = 20 * np.log10(rms)
    loudness_change_dB = target_dB - loudness
    y_normalized = y * (10 ** (loudness_change_dB / 20))
    return y_normalized

### Pre-processing : MFCC & DeltaMFCC

In [12]:
def get_features(df, train_mode=True):
    features = []
    labels = []
    for i, (index, row) in enumerate(tqdm(df.iterrows(), total=len(df)), 1):
        try:
            y, sr = librosa.load(os.path.join(CONFIG.ROOT_DIR, row['path']), sr=CONFIG.SR)
            
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CONFIG.N_MFCC)
            mfcc_resize = tf.image.resize(mfcc[:,:,np.newaxis], (40,40)).numpy()
            
            features.append(mfcc_resize)
            
            if train_mode:
                label = row['label']
                label_vector = np.zeros(CONFIG.N_CLASSES, dtype=float)
                label_vector[0 if label == 'fake' else 1] = 1
                labels.append(label_vector)
                
        except Exception as e:
            print(f'Error while {index} : {e}')
            continue
    
    if train_mode:
        return features, labels
    return features
        

In [13]:
train_features, train_labels = get_features(train)
val_features, val_labels = get_features(val)

100%|██████████| 11088/11088 [02:43<00:00, 67.63it/s]


In [15]:
print(train_features[10].shape)
print(len(train_features))
print(len(train_labels))

(40, 40, 1)
44350
44350


In [24]:
def save_np():
    os.makedirs(os.path.join(CONFIG.ROOT_DIR, 'npy'), exist_ok=True)
    np.save(os.path.join(CONFIG.ROOT_DIR, 'npy/train_features_CNN.npy'), train_features)
    np.save(os.path.join(CONFIG.ROOT_DIR, 'npy/train_labels_CNN.npy'), train_labels)
    np.save(os.path.join(CONFIG.ROOT_DIR, 'npy/val_features_CNN.npy'), val_features)
    np.save(os.path.join(CONFIG.ROOT_DIR, 'npy/val_labels_CNN.npy'), val_labels)

In [25]:
def load_np():
    train_features = np.load(os.path.join(CONFIG.ROOT_DIR, 'npy/train_features_CNN.npy'))
    train_labels = np.load(os.path.join(CONFIG.ROOT_DIR, 'npy/train_labels_CNN.npy'))
    val_features = np.load(os.path.join(CONFIG.ROOT_DIR, 'npy/val_features_CNN.npy'))
    val_labels = np.load(os.path.join(CONFIG.ROOT_DIR, 'npy/val_labels_CNN.npy'))
    
    return train_features, train_labels, val_features, val_labels

In [26]:
save_np()
# train_features, train_labels, val_features, val_labels = load_np()

In [17]:
train_features = np.array(train_features)
train_labels = np.array(train_labels)
val_features = np.array(val_features)
val_labels = np.array(val_labels)

In [18]:
print(train_features.shape, len(train_labels))
print(val_features.shape, len(val_labels))

(44350, 40, 40, 1) 44350
(11088, 40, 40, 1) 11088


### Dataset

In [20]:
class CustomDataset(Dataset):
    def __init__(self, mfcc, label):
        self.mfcc = mfcc
        self.label = label

    def __len__(self):
        return len(self.mfcc)

    def __getitem__(self, index):
        if self.label is not None:
            return self.mfcc[index], self.label[index]
        return self.mfcc[index]

In [21]:
train_dataset = CustomDataset(train_features, train_labels)
val_dataset = CustomDataset(val_features, val_labels)

In [23]:
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

### Define Model

In [48]:
class CNNModel(nn.Module):
    def __init__(self, output_dim=CONFIG.N_CLASSES):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.fc1 = nn.Linear(128*5*5, 128)
        self.bn4 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, output_dim)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = x.permute(0, 3, 1, 2)
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.bn4(self.fc1(x)))
        x = self.dropout(x)
        x = self.sigmoid(self.fc2(x))
        return x

### Train & Validation

In [35]:
from sklearn.metrics import roc_auc_score
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

In [36]:
def train(model, scheduler, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, CONFIG.N_EPOCHS+1):
        model.train()
        train_loss = []
        for features, labels in tqdm(iter(train_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            optimizer.zero_grad()
            
            output = model(features)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val AUC : [{_val_score:.5f}] LEARNING RATE : [{optimizer.param_groups[0]["lr"]:.5f}]')

        scheduler.step(_val_loss)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

In [37]:
def multiLabel_AUC(y_true, y_scores):
    auc_scores = []
    for i in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score

In [38]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss, all_labels, all_probs = [], [], []
    
    with torch.no_grad():
        for features, labels in tqdm(iter(val_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            probs = model(features)
            
            loss = criterion(probs, labels)

            val_loss.append(loss.item())

            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())
        
        _val_loss = np.mean(val_loss)

        all_labels = np.concatenate(all_labels, axis=0)
        all_probs = np.concatenate(all_probs, axis=0)
        
        # Calculate AUC score
        auc_score = multiLabel_AUC(all_labels, all_probs)
    
    return _val_loss, auc_score

### Run

In [49]:
model = CNNModel()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CONFIG.LEARNING_RATE)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

infer_model = train(model, scheduler, optimizer, train_loader, val_loader, device)

100%|██████████| 116/116 [00:00<00:00, 219.25it/s]


Epoch [1], Train Loss : [0.09083] Val Loss : [0.03037] Val AUC : [0.99912] LEARNING RATE : [0.00100]


100%|██████████| 116/116 [00:00<00:00, 289.63it/s]


Epoch [2], Train Loss : [0.02782] Val Loss : [0.02525] Val AUC : [0.99977] LEARNING RATE : [0.00100]


100%|██████████| 116/116 [00:00<00:00, 293.07it/s]


...

Epoch [48], Train Loss : [0.00007] Val Loss : [0.00375] Val AUC : [0.99999] LEARNING RATE : [0.00000]


100%|██████████| 116/116 [00:00<00:00, 274.13it/s]


Epoch [49], Train Loss : [0.00010] Val Loss : [0.00367] Val AUC : [0.99999] LEARNING RATE : [0.00000]


100%|██████████| 116/116 [00:00<00:00, 290.03it/s]

Epoch [50], Train Loss : [0.00010] Val Loss : [0.00375] Val AUC : [0.99999] LEARNING RATE : [0.00000]


### Inference

In [52]:
test = pd.read_csv(os.path.join(CONFIG.ROOT_DIR, 'test.csv'))
test_mfcc = get_features(test, False)
test_dataset = CustomDataset(test_mfcc, None)
test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

100%|██████████| 50000/50000 [16:15<00:00, 51.27it/s]


In [41]:
# np.save(os.path.join(CONFIG.ROOT_DIR, 'npy/test_mfcc.npy'), test_mfcc)
# test_mfcc = np.load(os.path.join(CONFIG.ROOT_DIR, 'npy/test_mfcc.npy'))

In [53]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for features in tqdm(iter(test_loader)):
            features = features.float().to(device)
            
            probs = model(features)

            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

In [54]:
preds = inference(infer_model, test_loader, device)

100%|██████████| 521/521 [00:01<00:00, 293.09it/s]


## Submission

In [55]:
submit = pd.read_csv(os.path.join(CONFIG.ROOT_DIR,'./sample_submission.csv'))
submit.iloc[:, 1:] = preds
submit.head()

,id,fake,real
0,TEST_00000,2.389369e-04,0.999771
1,TEST_00001,2.959154e-08,1.000000
2,TEST_00002,1.136608e-06,0.999999
3,TEST_00003,1.019144e-03,0.999075
4,TEST_00004,1.308857e-13,1.000000


In [56]:
submit.to_csv(f'./output/submit_RawCNN.csv', index=False)